# Revision notes from Enthuware Mock Test 4

## Inheritance of *state*, *type*, *implementation*
- *Inheritace of state* refers to the inheritance of instance fields/variables.

- *Inheritance of implementation* refers to the inheritance of instance methods.

- *Inheritance of types* refers to Java's ability to inherit from multiple *interfaces* and/or extend from multiple *classes*.

## You cannot throw a checked exception from a `catch` block unless the catch itself is nested within a `try` or...
The method which holds the `catch` is declared to throw the checked exception.

In [1]:
try {
    throw new Exception();
}

catch (Exception e) {
    throw new Exception();
}

ERROR:  java.lang.Exception

## Wrapper class constructors will not automatically convert larger/greater primitives
For example, the constructor for `Short` expects a `short` primitve and will throw an error if you pass an`int` unless you cast it to a `short` first. The principle is the same as when trying to assign an `int` primitive to a `short` variable.

In [9]:
// Short constructor with an int
Short s = new Short(7);

no suitable constructor found for Short(int): no suitable constructor found for Short(int)

In [10]:
// Short constructor with an int cast to a short
Short s = new Short((short)7);

null

## A reference to static field only initializes the class which declares that static field
This is a tricky one; basically if you have a static field which is inherited by a child class and then during program execution that static field is referred to via the child class (e.g. `ChildClass.staticField`) this will NOT cause intialization of the child class since in this case the child class didn't declare the static field (it inherited it).

So, a question about this is testing your knowledge of when/what causes initialization of a class.

In [16]:
package test.beaker;

class Parent { 
    static { System.out.println("Parent initialized");}
    static String nickName; // we're declaring a static field 'nickName' in the Parent class which will be inherited by any children
}

class Mammy extends Parent { // Mammy extends Parent and so inherits the field 'nickName'
    static {
        System.out.println("Mammy initialized");
        nickName = "Mammy"; // here we're using a static initializer block to set nickName to 'Mammy'
    } 
}

test.beaker.Parent

In [19]:
package test.beaker;

System.out.println(Mammy.nickName); // prints null

// EXPLANATION: since referencing a static field only initializes the class which DECLARES that field
// only Parent is initialized which is why we see 'Parent initialized' but not 'Mammy initialized'
// Java essentially looks at 'nickName' and sees it is "static" so it goes straight to where it was declared
// and initializes that class so that the 'nickName' field exists. Mammy never gets initialized, thus we never set the value of 
// nickName to 'Mammy'

null


null

Q18